In [1]:
import pandas as pd
import bs4 as bs
import requests


In [1]:
import requests
from bs4 import BeautifulSoup
import re

# URLs to scrape
urls = [
    "https://boardgamegeek.com/browse/boardgame/page/1?sort=numvoters&sortdir=desc",
    "https://boardgamegeek.com/browse/boardgame/page/2?sort=numvoters&sortdir=desc"
]

ids = []

def scrape_and_save(urls, filename="output.txt"):
    with open(filename, "w") as file:
        for url in urls:
            # Make a request to the webpage
            response = requests.get(url)
            if response.status_code == 200:
                # Parse the content with BeautifulSoup
                soup = BeautifulSoup(response.content, 'html.parser')
                # Find all <a> tags with class "primary"
                primary_links = soup.find_all('a', class_='primary')
                # Extract the board game ID from the href attribute
                for link in primary_links:
                    match = re.search(r'/boardgame/(\d+)/', link['href'])
                    if match:
                        file.write(match.group(1) + "\n")
                        ids.append(int(match.group(1)))
            else:
                print(f"Failed to retrieve data from {url}")

# Call the function
scrape_and_save(urls)


In [2]:
ids

[13,
 822,
 30549,
 68448,
 167791,
 173346,
 266192,
 178900,
 36218,
 230802,
 9209,
 169786,
 148228,
 14996,
 31260,
 40692,
 3076,
 70323,
 129622,
 163412,
 2651,
 39856,
 84876,
 174430,
 110327,
 28143,
 199792,
 478,
 161936,
 34635,
 54043,
 162886,
 237182,
 65244,
 244521,
 1927,
 12333,
 183394,
 98778,
 120677,
 204583,
 170216,
 133473,
 131357,
 11,
 50,
 150376,
 10547,
 224517,
 316554,
 147020,
 70919,
 312484,
 181304,
 342942,
 121921,
 205637,
 201808,
 284083,
 157354,
 199561,
 41114,
 193738,
 124361,
 463,
 15987,
 126163,
 194655,
 295947,
 2655,
 171,
 37111,
 146021,
 205059,
 1406,
 181,
 157969,
 192291,
 102794,
 103343,
 172225,
 291457,
 31481,
 40834,
 128882,
 6249,
 96848,
 143884,
 39463,
 9220,
 320,
 187645,
 158899,
 3955,
 132531,
 35677,
 182028,
 244992,
 148949,
 164928,
 172,
 136063,
 233867,
 167355,
 10630,
 124742,
 263918,
 432,
 18602,
 2223,
 236457,
 188834,
 93,
 42,
 177736,
 16992,
 50381,
 72125,
 147949,
 254640,
 146508,
 1506

In [3]:
# Create a DataFrame
#data of the board games

import requests
import pandas as pd
import xml.etree.ElementTree as ET

def fetch_game_data(game_id):
    """Fetch game data for a given game ID."""
    url = f'https://boardgamegeek.com/xmlapi2/thing?id={game_id}&stats=1'
    response = requests.get(url)
    while response.status_code != 200:
        response = requests.get(url)  # Retry fetching until success
    return response.text

def parse_game_data(game_xml):
    """Parse XML data for a single game."""
    root = ET.fromstring(game_xml)
    game_info = {}
    for item in root.findall('item'):
        game_info['id'] = item.get('id')
        game_info['name'] = item.find('name').get('value')
        statistics = item.find('statistics')
        ratings = statistics.find('ratings')
        game_info['average_rating'] = ratings.find('average').get('value')
        game_info['Geek_rating'] = ratings.find('bayesaverage').get('value')
        game_info['rank'] = ratings.find('ranks/rank').get('value')
        game_info['num_voters'] = ratings.find('usersrated').get('value')
        
        #Rank and genres and number of voters
        types = item.find('statistics').find('ratings').find('ranks').findall('rank')
        types = types[1]
        game_info['type'] = types.get('name')
        game_info['category'] = item.findall('link')[0].get('value')
            
        
        # Additional fields
        game_info['playing_time'] = item.find('playingtime').get('value')
        

    return game_info

def main():
    games_data = []

    for game_id in ids:
        game_xml = fetch_game_data(game_id)
        game_data = parse_game_data(game_xml)
        games_data.append(game_data)
        print(f"Fetched and parsed game ID: {game_id}")

    df = pd.DataFrame(games_data)
    print(df.head())
    df.to_csv('top200.csv', index=False)

if __name__ == '__main__':
    main()



Fetched and parsed game ID: 13
Fetched and parsed game ID: 822
Fetched and parsed game ID: 30549
Fetched and parsed game ID: 68448
Fetched and parsed game ID: 167791
Fetched and parsed game ID: 173346
Fetched and parsed game ID: 266192
Fetched and parsed game ID: 178900
Fetched and parsed game ID: 36218
Fetched and parsed game ID: 230802
Fetched and parsed game ID: 9209
Fetched and parsed game ID: 169786
Fetched and parsed game ID: 148228
Fetched and parsed game ID: 14996
Fetched and parsed game ID: 31260
Fetched and parsed game ID: 40692
Fetched and parsed game ID: 3076
Fetched and parsed game ID: 70323
Fetched and parsed game ID: 129622
Fetched and parsed game ID: 163412
Fetched and parsed game ID: 2651
Fetched and parsed game ID: 39856
Fetched and parsed game ID: 84876
Fetched and parsed game ID: 174430
Fetched and parsed game ID: 110327
Fetched and parsed game ID: 28143
Fetched and parsed game ID: 199792
Fetched and parsed game ID: 478
Fetched and parsed game ID: 161936
Fetched and

In [15]:
import xml.etree.ElementTree as ET

In [67]:
url = f'https://boardgamegeek.com/xmlapi2/thing?id=161936&stats=1'

game_xml = requests.get(url).text
print(game_xml)

root = ET.fromstring(game_xml)

items = root.findall('item')

for item in items:
    print(item.find('name').get('value'))
    print(item.get('id'))
    print(item.find('statistics').find('ratings').find('average').get('value'))
    print(item.find('statistics').find('ratings').find('ranks').find('rank').get('value'))
    ranks = item.find('statistics').find('ratings').find('ranks').findall('rank')
    cats = item.findall('link')[0].get('value')
    print(cats)
    
    

<?xml version="1.0" encoding="utf-8"?><items termsofuse="https://boardgamegeek.com/xmlapi/termsofuse"><item type="boardgame" id="161936">
         <thumbnail>https://cf.geekdo-images.com/-Qer2BBPG7qGGDu6KcVDIw__thumb/img/NQQcjS31TO0DE246N9rpt0hd9eo=/fit-in/200x150/filters:strip_icc()/pic2452831.png</thumbnail>
      <image>https://cf.geekdo-images.com/-Qer2BBPG7qGGDu6KcVDIw__original/img/PlzAH7swN1nsFxOXbfUvE3TkE5w=/0x0/filters:format(png)/pic2452831.png</image>
                                     				
				<name type="primary" sortindex="1" value="Pandemic Legacy: Season 1" />
			
						                               				
				<name type="alternate" sortindex="1" value="Pandemic Legacy:  Saison 1" />
			    				
				<name type="alternate" sortindex="1" value="Pandemic Legacy: 1. Évad" />
			    				
				<name type="alternate" sortindex="1" value="Pandemic Legacy: 1a. Temporada" />
			    				
				<name type="alternate" sortindex="1" value="Pandemic Legacy: Rok 1" />
			    				
				<na